In [1]:
import numpy as np
import sympy
from sympy.tensor.tensor import TensorIndexType, TensorHead, tensor_indices
from tqdm import tqdm

sympy.init_printing()

In [2]:
A, B, C, D, E, F, G, H, I = sympy.symbols("A B C D E F G H I", real=True)
F_0123 = sympy.symbols("F_{0123}")

In [3]:
g_mat = sympy.Matrix([
    [A + sympy.I * B, C + sympy.I * D, 0, 0],
    [C + sympy.I * D, E + sympy.I * F, 0, 0],
    [0, 0, G + sympy.I * H, 0],
    [0, 0, 0, I]
])
g_mat

⎡A + ⅈ⋅B  C + ⅈ⋅D     0     0⎤
⎢                            ⎥
⎢C + ⅈ⋅D  E + ⅈ⋅F     0     0⎥
⎢                            ⎥
⎢   0        0     G + ⅈ⋅H  0⎥
⎢                            ⎥
⎣   0        0        0     I⎦

In [4]:
F_tensor = [
    # F_0XXX
    [
        # F_00XX
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
        # F_01XX
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, F_0123],
         [0, 0, -F_0123, 0]],
        # F_02XX
        [[0, 0, 0, 0],
         [0, 0, 0, -F_0123],
         [0, 0, 0, 0],
         [0, F_0123, 0, 0]],
        # F_03XX
        [[0, 0, 0, 0],
         [0, 0, F_0123, 0],
         [0, -F_0123, 0, 0],
         [0, 0, 0, 0]]
    ],
    # F_1XXX
    [
        # F_10XX
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, -F_0123],
         [0, 0, F_0123, 0]],
        # F_11XX
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
        # F_12XX
        [[0, 0, 0, F_0123],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [-F_0123, 0, 0, 0]],
        # F_13XX
        [[0, 0, -F_0123, 0],
         [0, 0, 0, 0],
         [F_0123, 0, 0, 0],
         [0, 0, 0, 0]]
    ],
    # F_2XXX
    [
        # F_20XX
        [[0, 0, 0, 0],
         [0, 0, 0, F_0123],
         [0, 0, 0, 0],
         [0, -F_0123, 0, 0]],
        # F_21XX
        [[0, 0, 0, -F_0123],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [F_0123, 0, 0, 0]],
        # F_22XX
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
        # F_23XX
        [[0, F_0123, 0, 0],
         [-F_0123, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]
    ],
    # F_3XXX
    [
        # F_30XX
        [[0, 0, 0, 0],
         [0, 0, -F_0123, 0],
         [0, F_0123, 0, 0],
         [0, 0, 0, 0]],
        # F_31XX
        [[0, 0, F_0123, 0],
         [0, 0, 0, 0],
         [-F_0123, 0, 0, 0],
         [0, 0, 0, 0]],
        # F_32XX
        [[0, -F_0123, 0, 0],
         [F_0123, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],
        # F_33XX
        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]
    ]
]
F_tensor

In [21]:
# Lorentzian = TensorIndexType('Lorentzian', dummy_name='L')
# i0, i1, i2, i3, i4, i5, i6, i7 = tensor_indices('i0:8', Lorentzian)
# j0, j1, j2, j3, j4, j5, j6, j7 = tensor_indices('j0:8', Lorentzian)
# g4FF_mat = sympy.tensorproduct(sympy.tensorproduct(g_mat, sympy.tensorproduct(sympy.tensorproduct(g_mat, g_mat), g_mat)), sympy.tensorproduct(F_tensor, F_tensor))
# g4FF_tensor = TensorHead('g4FF', [Lorentzian] * 16)
# repl = {g4FF_tensor(i0, j0, i1, j1, i2, j2, i3, j3, -i4, -i5, -i6, -i7, -j4, -j5, -j6, -j7): g4FF_mat}
# quadratic_form = g4FF_tensor(i0, j0, i1, j1, i2, j2, i3, j3, -i0, -i1, -i2, -i3, -j0, -j1, -j2, -j3).replace_with_arrays(repl)

In [6]:
indices_perm = [
    # 0XXX
    ([0, 1, 2, 3], 1),
    ([0, 1, 3, 2], -1),
    ([0, 2, 1, 3], -1),
    ([0, 2, 3, 1], 1),
    ([0, 3, 1, 2], 1),
    ([0, 3, 2, 1], -1),
    # 1XXX
    ([1, 0, 2, 3], -1),
    ([1, 0, 3, 2], 1),
    ([1, 2, 0, 3], 1),
    ([1, 2, 3, 0], -1),
    ([1, 3, 0, 2], -1),
    ([1, 3, 2, 0], 1),
    # 2XXX
    ([2, 0, 1, 3], 1),
    ([2, 0, 3, 1], -1),
    ([2, 1, 0, 3], -1),
    ([2, 1, 3, 0], 1),
    ([2, 3, 0, 1], 1),
    ([2, 3, 1, 0], -1),
    # 3XXX
    ([3, 0, 1, 2], -1),
    ([3, 0, 2, 1], 1),
    ([3, 1, 0, 2], 1),
    ([3, 1, 2, 0], -1),
    ([3, 2, 0, 1], -1),
    ([3, 2, 1, 0], 1)
]

In [12]:
quadratic_form = 0

for indices_1, F_val_1 in indices_perm:
    for indices_2, F_val_2 in indices_perm:
        i1, i2, i3, i4 = indices_1
        j1, j2, j3, j4 = indices_2
        quadratic_form += g_mat[i1, j1] * g_mat[i2, j2] * g_mat[i3, j3] * g_mat[i4, j4] * F_val_1 * F_val_2

In [11]:
quadratic_form

Take the real part.

In [22]:
(quadratic_form.expand().collect(sympy.I) - quadratic_form.expand().collect(sympy.I).coeff(sympy.I) * sympy.I)

It is the same as the real part of the determinant.

In [20]:
g_mat.det().expand().collect(sympy.I)